In [10]:
import sqlite3
con = sqlite3.connect("SQL_5.db")
cur = con.cursor()

In [11]:
cur.execute("DROP TABLE IF EXISTS Pieces")
cur.execute("DROP TABLE IF EXISTS Providers")
cur.execute("DROP TABLE IF EXISTS Provides")

In [12]:
cur.execute("""
CREATE TABLE Pieces (
 Code INTEGER NOT NULL,
 Name TEXT NOT NULL,
 PRIMARY KEY (Code)
 );
 """)

cur.execute("""
CREATE TABLE Providers (
 Code VARCHAR(40) NOT NULL,  
 Name TEXT NOT NULL,
 PRIMARY KEY (Code) 
 );
 """)

cur.execute("""
CREATE TABLE Provides (
 Piece INTEGER, 
 Provider VARCHAR(40), 
 Price INTEGER NOT NULL,
 PRIMARY KEY(Piece, Provider) 
 );
 """)

In [13]:
cur.executescript("""
INSERT INTO Providers(Code, Name) VALUES('HAL','Clarke Enterprises');
INSERT INTO Providers(Code, Name) VALUES('RBT','Susan Calvin Corp.');
INSERT INTO Providers(Code, Name) VALUES('TNBC','Skellington Supplies');

INSERT INTO Pieces(Code, Name) VALUES(1,'Sprocket');
INSERT INTO Pieces(Code, Name) VALUES(2,'Screw');
INSERT INTO Pieces(Code, Name) VALUES(3,'Nut');
INSERT INTO Pieces(Code, Name) VALUES(4,'Bolt');

INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'HAL',10);
INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'HAL',20);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'TNBC',14);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'RBT',50);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'TNBC',45);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'HAL',5);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'RBT',7);
""")

In [14]:
def run(cmd):
    r = cur.execute(cmd)
    display(r.fetchall())

## 5.1 Select the name of all the pieces. 

In [15]:
run("""
SELECT Name 
FROM Pieces
;""")

[('Sprocket',), ('Screw',), ('Nut',), ('Bolt',)]

##  5.2  Select all the providers' data. 

In [16]:
run("""
SELECT * 
FROM Providers
;""")

[('HAL', 'Clarke Enterprises'),
 ('RBT', 'Susan Calvin Corp.'),
 ('TNBC', 'Skellington Supplies')]

## 5.3 Obtain the average price of each piece (show only the piece code and the average price).

In [19]:
run("""
SELECT Piece, ROUND(AVG(Price),2) 
FROM Provides
GROUP BY Piece 
;""")

[(1, 12.5), (2, 16.33), (3, 47.5), (4, 6.0)]

## 5.4  Obtain the names of all providers who supply piece 1.

In [22]:
run("""
SELECT Providers.Name 
FROM Provides
JOIN Providers
ON Providers.Code = Provides.Provider
WHERE Piece = 1
;""")

[('Clarke Enterprises',), ('Susan Calvin Corp.',)]

## 5.5 Select the name of pieces provided by provider with code "HAL".

In [30]:
run("""
SELECT Pieces.Name 
FROM Pieces
JOIN Provides
ON Pieces.Code = Provides.Piece
WHERE Provides.Provider = 'HAL'
;""")

[('Sprocket',), ('Screw',), ('Bolt',)]

## 5.6 For each piece, find the most expensive offering of that piece and include the piece name, provider name, and price (note that there could be two providers who supply the same piece at the most expensive price).

In [38]:
run("""
SELECT Pieces.Name, Providers.Name, Provides.Price
FROM Pieces
JOIN Provides ON Pieces.Code = Provides.Piece
JOIN Providers ON Providers.Code = Provides.Provider
WHERE Provides.Price = (
  SELECT MAX(Price)
  FROM Provides
  WHERE Provides.Piece = Pieces.Code
);""")

[('Sprocket', 'Susan Calvin Corp.', 15),
 ('Screw', 'Clarke Enterprises', 20),
 ('Nut', 'Susan Calvin Corp.', 50),
 ('Bolt', 'Susan Calvin Corp.', 7)]

## 5.7 Add an entry to the database to indicate that "Skellington Supplies" (code "TNBC") will provide sprockets (code "1") for 7 cents each.

In [39]:
run("""
INSERT INTO Provides (Piece, Provider, Price)
VALUES (1, 'TNBC', 7)
;
""")

[]

## 5.8 Increase all prices by one cent.

In [40]:
run("""
UPDATE Provides 
SET Price = Price + 1
;
""")

[]

## 5.9 Update the database to reflect that "Susan Calvin Corp." (code "RBT") will not supply bolts (code 4).

In [41]:
run("""
DELETE FROM Provides 
WHERE Piece = 4 AND Provider = 'RBT'
;
""")

[]

In [42]:
run("""
SELECT * 
FROM Provides
;""")

[(1, 'HAL', 11),
 (1, 'RBT', 16),
 (2, 'HAL', 21),
 (2, 'RBT', 16),
 (2, 'TNBC', 15),
 (3, 'RBT', 51),
 (3, 'TNBC', 46),
 (4, 'HAL', 6),
 (1, 'TNBC', 8)]

## 5.10 Update the database to reflect that "Susan Calvin Corp." (code "RBT") will not supply any pieces (the provider should still remain in the database).

In [43]:
run("""
DELETE FROM Provides 
WHERE Provider = 'RBT'
;
""")

[]

In [44]:
run("""
SELECT * 
FROM Provides
;""")

[(1, 'HAL', 11),
 (2, 'HAL', 21),
 (2, 'TNBC', 15),
 (3, 'TNBC', 46),
 (4, 'HAL', 6),
 (1, 'TNBC', 8)]

In [45]:
con.close()